# Tamanho do Mercado das Usinas Sucroalcooleiras

# Setup Inicial

Carregando linhas com configurações iniciais já escritas em notebook base

No bloco de comando "%run" não pode ter nenhum outra linha de comando ou comentário

In [1]:
%run nb00_Setup_Usinas{}

# Leitura dos Dados 

In [2]:
usinas   = pd.read_parquet(abfss_path_enriched + 'usinas/usi02_tancagens_impt_usinas.parquet',
                            storage_options = {'linked_service' : linked_service_enriched})
hist_usinas = pd.read_parquet(abfss_path_enriched + 'usinas/usi03_vendas_impt_mes.parquet',
                            storage_options = {'linked_service' : linked_service_enriched})

prop  = pd.read_parquet(abfss_path_enriched + 'volumetria/vol01_prop_consfinal_vegetativo.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
cf  = pd.read_parquet(abfss_path_enriched + 'volumetria/vol01_vendas_vibra_mes.parquet',
                        storage_options = {'linked_service' : linked_service_enriched})
diesel_mes = pd.read_parquet(abfss_path_enriched + 'anp/anp_diesel_mes.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

# Tamanho do Mercado

O tamanho do mercado pode ser obtido por duas manairas:

1. Pela visão da Clientela</br>
    - $TM = C + \bar{C} $  
2. Pela visão da participação da Atividade Econômica no Segmento Consumidor final</br>
    - $TM = U_{part} \times Cons.Final$

Onde assumiu-se que:</br>
- $Cons.Final = Cons.Final_{Global}$ e</br>
- $U_{part} = U_{part: Vibra}$

In [3]:
tm1_m = hist_usinas.groupby(['Data','IMP'])['Vol_Total'].sum().reset_index()
tm1_m = tm1_m.pivot_table(values='Vol_Total', index='Data', columns='IMP').reset_index()
tm1_m.rename_axis(None, axis=1, inplace=True)
tm1_m.columns = ['Data','Vol_Cliente', 'Vol_NaoCliente']
tm1_m['Vol_Usina'] = tm1_m[['Vol_Cliente', 'Vol_NaoCliente']].sum(axis=1)

In [4]:
hist = hist_usinas.groupby(['Data','IMP'])['Vol_Total'].agg('sum').reset_index()
# hist['ds'] = pd.to_datetime(hist['ANOMES'],format='%Y-%m' )
# hist['y']= hist['Vol_Total']
# hist = hist[['ds','y']]

In [5]:
tm1_m

In [6]:
tm2_m = pd.DataFrame(diesel_mes.loc['2016':]['consumidor final']*prop['USINA'][:-1]).reset_index()
tm2_m.columns = ['Data','Vol_Usina']

In [7]:
tm2_m

# Teste de Aderência das duas séries

In [8]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.kolmogorov.html

x = tm1_m['Vol_Usina'][:85].values
y = tm2_m['Vol_Usina'][:85].values

# probabilidades
px = x/x.sum()
py = y/y.sum()

#probabilidades acumuladas
pxc = px.cumsum()
pyc = py.cumsum()


grid = np.linspace(-3, 3, 85)

plt.step(grid, px.cumsum(), label='Tamanho de Mercado 1')
plt.plot(grid, py.cumsum(), label='Tamanho de Mercado 2')
plt.ylim([0, 1]); plt.grid(True); plt.legend();

gaps = np.column_stack([pyc-pxc, pxc-pyc])
a, b = np.argmax(gaps, axis=0)
plt.vlines([grid[a]], pyc[a], pxc[a], color='r', linestyle='dashed', lw=4)
plt.vlines([grid[b]], pxc[b], pyc[b], color='r', linestyle='dashed', lw=4)

#plt.show()

In [9]:
Dn = np.max(gaps) # = pxc[b]- pyc[b]
Kn = np.sqrt(len(x)) * Dn
print('Dn=%f, sqrt(n)*Dn=%f' % (Dn, Kn))

In [10]:
stats.kstest(x,y, alternative='two-sided')

# Salvando Tamanho do Mercado

In [11]:
tm1_m.to_parquet(abfss_path_enriched + 'usinas/usi04_tamanho_mercado.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

# Gráficos

In [12]:
cores = ['green','gray','purple', 'gold']

fig = make_subplots(rows=1, cols=1,
                    y_title='Volume (m³)',
                    subplot_titles=(''),
                    shared_yaxes=False,
                    vertical_spacing=0.1 )

# grafico 01
for c,i in enumerate(tm1_m.columns[1:3]):
    fig.add_trace(go.Scatter(
                        name = i,
                        x = tm1_m['Data'],
                        y = tm1_m[i],
                        line = {'color': cores[c]},
                        visible=True,
                        stackgroup='one'),
                    row=1, col=1
    )

fig.add_trace(go.Scatter( # Linha do Crescimento Vegetativo Vibra -  Consumidor Final
            x = tm1_m['Data'],
            y = tm1_m['Vol_Usina'],
            line = {'color': 'green', 'dash':'dot'},
            name = 'TM (Cli + ÑCli)',
            showlegend=True),
    row=1, col=1)

fig.add_trace(go.Scatter( # Linha do Tamanho de Mercado (%Usi * ConsFinal)
            x = tm2_m['Data'],
            y = tm2_m['Vol_Usina'],
            line = dict(color='red', width=4),
            name = 'TM (%Usi * ConsFinal)',
            showlegend=True),
    row=1, col=1)

fig.update_xaxes(matches='x', rangeslider_visible=True, rangeslider_thickness = 0.05)
fig.update_xaxes(rangeslider= {'visible':True}, row=1, col=1)

fig.add_annotation(
            showarrow=False,
            text='Gerado em ' + dt.now(tz_SP).strftime("%d %b %Y às %H:%M:%S"),
            font=dict(size=12),
            align="right",
            x=1.18,
            y=-0.24,
            xref='paper',
            yref='paper',
            xanchor='right',
            yanchor='bottom',)

# titulo
fig.update_layout(
                template='plotly_white', # template ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
                title = 'Volume de Diesel Vendido para Usinas Sucroalcooleiras' + '<br><sup>'+ 
                        'Considerando total de usinas MAPA (mar/2023)'+'</sup>',
                legend_title = "Legenda",
                font=dict(
                        family = "Courier New, monospace",
                        size = 14,
                        color = "royalblue"),
                height=600
)

fig = plotly.offline.plot(fig, output_type='div')
# 
displayHTML(fig)


In [13]:
def agrup_graf(df, geo='UF'):
    aux = df.set_index('Data')
    aux = aux.groupby([geo])['Vol_Total'].resample('MS').sum()
    aux = aux.unstack(level=0)
    #aux.columns = aux.columns.droplevel(0)
    aux = aux.rename_axis(None, axis=1)
    return(aux)

In [14]:
titulo = ('Volume de Diesel Vendido para Usinas Sucroalcooleiras por região' + '<br><sup>'+
           'Considerando total de usinas MAPA (mar/2023)'+'</sup>')

# multi_plot(agrup_graf(hist_usinas, geo='REGIAO3'), titulo, 'Agrupamentos<br>Regionais')

In [15]:
df = tm1_m[['Data']]
df['Erro'] = (tm1_m['Vol_Usina'] - tm2_m['Vol_Usina'])/tm2_m['Vol_Usina']

fig = go.Figure()
fig.add_trace(go.Scatter( # Linha do Crescimento Vegetativo Vibra -  Consumidor Final
            x = df['Data'],
            y = df['Erro'],
            line = {'color': 'green', 'dash':'dot'}))
fig.update_layout(
                template='plotly_white', # template ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]
                title = 'Erro Relativo Mensal em relação do Tamanho do Mercado Proporcional',
                font=dict(
                        family = "Courier New, monospace",
                        size = 14,
                        color = "royalblue"),
                height=400
)
fig = plotly.offline.plot(fig, output_type='div')
displayHTML(fig)